# Base Pytorch Model Implementation

#### More Tests

In [ ]:
# copied from above.

PATH = "/content/drive/My Drive/model.pth"

model = models.efficientnet_b0(weights='DEFAULT')

for val in model.features.parameters():
      val.requires_grad = False

model.classifier = nn.Sequential(
    nn.Linear(model.classifier[1].in_features, 1024),
    nn.ReLU(),
    nn.Linear(1024, len(trainset.classes))
)

model.load_state_dict(torch.load(PATH))
model.eval()

# parameters setting
loss_function = nn.CrossEntropyLoss()
proj_norm=2
lr=0.001
radius=0.01
data = next(iter(trainloader))

adv_features = general_PGD(model, loss_function, data, proj_norm=2, radius=0.03, lr=0.01, steps=10)

#### Now, we'll try some different attacks (including some random ones), and see which ones perform well

In [ ]:
eps = 0.03 # the adversarial tolerance

# FGSM (Fast Gradient Sign Method) attack
fgsm_attack = FGSM(model, eps=eps)

# Gaussian noise attack
gaussian_noise_attack = GN(model, std=0.1)

# Jitter attack
jitter_attack = Jitter(model, eps=eps, steps=10)

adv_images_fgsm, adv_images_gaussian,  adv_images_jitter, adv_images_pgd =\
 None, None, None, adv_features
# Iterate through the test loader
images, labels = data
# Apply attacks
adv_images_fgsm = fgsm_attack(images, labels)
adv_images_gaussian = gaussian_noise_attack(images, labels)
adv_images_jitter = jitter_attack(images, labels)



In [ ]:
adv_images_nothing = images

#### Finally, we'll evaluate how accurately the model can predict different adversarial inputs.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

# Define a custom dataset to hold the adversarial examples
class AdversarialDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]





In [ ]:
# Define a function to evaluate model accuracy
def evaluate_model(model, dataloader):
    device = next(model.parameters()).device
    correct = 0
    total = 0

    # Set model to evaluation mode
    model.eval()

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images).to(device)

            # Get predicted labels
            _, predicted = torch.max(outputs, 1)

            # Update total and correct predictions
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate accuracy
    accuracy = (correct / total) * 100
    print('Accuracy on adversarial examples: {:.2f}%'.format(accuracy))



Accuracy on adversarial examples: 12.50%


In [ ]:
attacks = ["none", "fgsm", "gaussian noise", "jitter", "pgd"]

for attack, adv_data in zip(attacks, [adv_images_nothing, adv_images_fgsm, adv_images_gaussian, adv_images_jitter, adv_images_pgd]):
  print(f"Adversarial Attack with attack={attack}")
  # Convert adversarial examples to a dataset
  adversarial_dataset = AdversarialDataset(adv_data, labels)

  # Define a DataLoader for the adversarial examples
  adversarial_loader = DataLoader(adversarial_dataset, batch_size=64, shuffle=False)
  evaluate_model(model, adversarial_loader)

Adversarial Attack with attack=none
Accuracy on adversarial examples: 1.56%
Adversarial Attack with attack=fgsm
Accuracy on adversarial examples: 12.50%
Adversarial Attack with attack=gaussian noise
Accuracy on adversarial examples: 1.56%
Adversarial Attack with attack=jitter
Accuracy on adversarial examples: 0.00%
Adversarial Attack with attack=pgd
Accuracy on adversarial examples: 1.56%
